In [35]:
#IMPORTS put all here
import random
import matplotlib.pyplot as plt

from datetime import datetime

#tf imports for reading file
import numpy as np
import os
import PIL


import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow import keras


from tensorflow.keras.regularizers import l2

#extras
print(tf.__version__)
import pathlib
import shutil

2.8.0


In [36]:
#path to image dataset
spec_path = '../content/spectrograms6sec'
chroma_path='../content/chroma6sec'
test_path='../content/testset'
genre_list=os.listdir(spec_path)


In [37]:
# os.mkdir('../content/chroma6secnew')
# dest_path='../content/chroma6secnew'

# for genre in genre_list:
#   os.mkdir(f'../content/chroma6secnew/{genre}')



# #no of samples per class
# n=3000
# cnt=0

# random.seed(123)

# for genre in genre_list:
#   genre_path=os.path.join(chroma_path,genre)
#   dest_genre_path=os.path.join(dest_path,genre)

#   #shuffling and selecting 3000 samples
#   genre_songs=os.listdir(genre_path)
#   random.shuffle(genre_songs)
#   selected_spec=genre_songs[:n]
  
#   for song in selected_spec:
#       source=os.path.join(genre_path,song)
#       dest=os.path.join(dest_genre_path,song)
#       d= shutil.copyfile(source, dest)


# EQUALISING DATASET

In [38]:
#loading data_sets
spec_path='../content/spectrograms6secnew/'
chroma_path='../content/chroma6secnew/'

data_dir_spec=pathlib.Path(spec_path)
data_dir_chroma=pathlib.Path(chroma_path)


In [39]:
#adding dataset
batch_size=64
image_height=100
image_width=200


In [40]:
#training dataset spectrograms
train_ds_spec = tf.keras.utils.image_dataset_from_directory(
  data_dir_spec,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

# training dataset chromagrams

train_ds_chroma = tf.keras.utils.image_dataset_from_directory(
  data_dir_chroma,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 30000 files belonging to 10 classes.
Using 24000 files for training.
Found 30000 files belonging to 10 classes.
Using 24000 files for training.


In [41]:
val_ds_spec = tf.keras.utils.image_dataset_from_directory(
  data_dir_spec,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

val_ds_chroma = tf.keras.utils.image_dataset_from_directory(
  data_dir_chroma,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)


Found 30000 files belonging to 10 classes.
Using 6000 files for validation.
Found 30000 files belonging to 10 classes.
Using 6000 files for validation.


In [42]:
for image_batch, labels_batch in train_ds_spec:
  print(image_batch.shape)
  print(labels_batch.shape)
  break
for image_batch, labels_batch in train_ds_chroma:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 100, 200, 3)
(64,)
(64, 100, 200, 3)
(64,)


In [43]:
class_names = train_ds_spec.class_names


In [44]:
AUTOTUNE = tf.data.AUTOTUNE

#
train_ds_spec = train_ds_spec.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_spec = val_ds_spec.cache().prefetch(buffer_size=AUTOTUNE)


#
train_ds_chroma = train_ds_chroma.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_chroma = val_ds_chroma.cache().prefetch(buffer_size=AUTOTUNE)



In [45]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
os.mkdir('models')

In [ ]:
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_ds_spec,
    validation_data=val_ds_spec,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )


